# ER Calculation Modification

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from calculator import ERCalculator
cal = ERCalculator()

## 1 Aggregate OpMode

In [2]:
# load trajectory
read_path = 'data/2-mapmatched/matched/'
traj_files = os.listdir(read_path)
trajs = {}
for traj_file in traj_files:
    key = traj_file.split('.')[0]
    trajs[key] = pd.read_csv(read_path + traj_file)

In [11]:
traj_ei5 = pd.concat([trajs['Ei5-DRIVER1'], trajs['Ei5-DRIVER2']]).reset_index(drop=True)
traj_modely = pd.concat([trajs['ModelY-DRIVER1'], trajs['ModelY-DRIVER2']]).reset_index(drop=True)

In [44]:
def aggOpMode(traj):
    traj = cal.OpModeDetect(traj)
    traj['braking'] = traj.apply(lambda x: 1 if (x['trac_power[kW]'] < 0) & (x['pad_pressure'] >= 5) else 0, axis=1)
    opmodes = cal.calOpModeInfo(traj, accBins=np.arange(-4.5, 0.1, 0.1))
    opmodes = cal.calOpModeERs(opmodes, decelBins=np.arange(-4.5, 0.1, 0.1))
    return opmodes

In [45]:
opmodes_ei5 = aggOpMode(traj_ei5)
opmodes_modely = aggOpMode(traj_modely)

## 2 Comparison

In [ ]:
import pickle
with open(r"D:\OneDrive - 东南大学\5 我的代码\brake-wear-emission-modeling\data\agg\agg_OpMode.pkl", 'rb') as f:
    opmodes = pickle.load(f)
opmodes['brakeFrac'] = opmodes['brakeCount'] / opmodes['trajCount']
opmodes['brakeDecelBinCount'] = opmodes['brakeDecelBinCount'].apply(lambda x: x[::-1])
opmodes['brakeDecelBinProp'] = opmodes['brakeDecelBinCount'] / opmodes['brakeCount']
opmodes = cal.calOpModeERs(opmodes, decelBins=np.arange(-4.5, 0.1, 0.1))

### 2.1 OpMode Brake Frac

In [76]:
pd.concat([opmodes['brakeFrac'].rename('brakeFrac_chengdu'), opmodes_ei5['brakeFrac'].rename('brakeFrac_ei5'), opmodes_modely['brakeFrac'].rename('brakeFrac_modely')], axis=1).loc[[0,1,11,12,21,22,33]]

,brakeFrac_chengdu,brakeFrac_ei5,brakeFrac_modely
0,1.0,0.586708,0.029218
1,0.06373,0.028881,0.001963
11,0.936091,0.557491,0.01006
12,0.035069,0.111853,0.004317
21,0.983018,0.215909,0.001701
22,0.166791,0.016706,0.0
33,0.619789,0.015385,0.0


### 2.2 OpMode Brake Decel

In [87]:
for opmode in [opmodes, opmodes_ei5, opmodes_modely]:
    opmode['brakeDecelMean'] = opmode['brakeDecelBinProp'].apply(lambda x: sum(x * np.arange(-4.5, 0.0, 0.1)))
pd.concat([opmodes['brakeDecelMean'].rename('brakeDecelMean_chengdu'), opmodes_ei5['brakeDecelMean'].rename('brakeDecelMean_ei5'), opmodes_modely['brakeDecelMean'].rename('brakeDecelMean_modely')], axis=1).loc[[0,1,11,12,21,22,33]]

,brakeDecelMean_chengdu,brakeDecelMean_ei5,brakeDecelMean_modely
0,-1.690422,-0.856814,-1.380645
1,-0.190209,-0.245833,-0.250000
11,-0.489207,-0.664063,-1.040000
12,-0.169473,-0.486567,-0.866667
21,-0.521305,-0.684962,-0.700000
22,-0.219323,-0.314286,0.000000
33,-0.589040,-0.700000,0.000000


In [82]:
pd.concat([opmodes['ER[g/hr/veh]'].rename('ER_chengdu'), opmodes_ei5['ER[g/hr/veh]'].rename('ER_ei5'), opmodes_modely['ER[g/hr/veh]'].rename('ER_modely')], axis=1).loc[[0,1,11,12,21,22,33]]

,ER_chengdu,ER_ei5,ER_modely
0,4.430523,0.815062,0.305155
1,0.000058,0.000053,0.000004
11,0.035765,0.255182,0.015805
12,0.000018,0.010416,0.001213
21,0.040632,0.017625,0.000115
22,0.000197,0.000094,0.000000
33,0.033038,0.001230,0.000000


## Save OpMode ERs

In [89]:
OpModeERs_Ei5 = opmodes_ei5['ER[g/hr/veh]'].to_numpy()
np.save("data/OpModeERs_Ei5.npy", OpModeERs_Ei5)

OpModeERs_ModelY = opmodes_modely['ER[g/hr/veh]'].to_numpy()
np.save("data/OpModeERs_ModelY.npy", OpModeERs_ModelY)